In [1]:
'''
Source code for paper "Learning modular robot control policies" in Transactions on Robotics
MLP comparisons
Julian Whitman, Dec. 2022. 

Run MLP policy on robot. Currently set to shared_trunk policy, could be changed to hardware_conditioned.
The control inputs stripped to frame with (x y yaw) removed
Uses the trained policy from the velocity input

Uses previous observation as input for next action

'''

# load libraries
import torch
from robot_env import robot_env
import numpy as np
import os
import pgnn_control as pgnnc
from utils import to_tensors, combine_state, wrap_to_pi, rotate, create_control_inputs
from shared_MLP_policy import shared_trunk_policy
from shared_MLP_utils import get_in_out_lens

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# init env
# use the env to get the size of the inputs and outputs
env = robot_env(show_GUI = True)
env.reset_terrain()

    
# USE_JOY = True
USE_JOY = False
if USE_JOY:
    from vrjoystick import init_joystick, read
    joy = init_joystick()
    


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Logitech Logitech Dual Action


In [2]:
## select which policy file to load
folder = 'saved/mbrl_shared_trunk1'
PATH =  'shared_trunk_control_iter3.pt'
PATH = os.path.join(folder,PATH)

save_dict = torch.load( PATH, map_location=lambda storage, loc: storage)
urdf_names = save_dict['urdf_names']
print(urdf_names)
# select which robot to put in
urdf_name = 'wnwwnw'
urdf_name = 'llllll'

fd_input_lens, fd_output_lens, policy_input_lens,action_lens,limb_types = get_in_out_lens(urdf_names)

fd_input_lens_sums = [sum(s) for s in fd_input_lens]
fd_output_lens_sums = [sum(s) for s in fd_output_lens]
action_lens_sums = [sum(a) for a in action_lens]
policy_input_lens_sums = [sum(s) for s in policy_input_lens]
print('fd_input_lens_sums, action_lens_sums, policy_input_lens_sums,fd_output_lens_sums: ' + 
    str(fd_input_lens_sums) + ', ' +
    str(action_lens_sums) +', ' +
    str(policy_input_lens_sums) +', ' +
    str(fd_output_lens_sums))

state_dict= save_dict['state_dict'] 
n_hidden_layers = save_dict['n_hidden_layers'] 
hidden_layer_size = save_dict['hidden_layer_size']
goal_len =3

print(save_dict['comment'])

env.reset_robot(urdf_name=urdf_name, randomize_start=False)

attachments = env.attachments
modules_types = env.modules_types
print('attachments: ' + str(attachments))
print('modules_types: ' + str(modules_types))
n_modules = len(modules_types)

env_state_init = env.get_state()
module_state_len = []
for s in env_state_init:
    module_state_len.append(len(s))

state_len= np.sum(module_state_len)
action_len = env.num_joints
module_action_len = list(np.diff(env.action_indexes))

module_sa_len = module_state_len+ module_action_len


policy_network = shared_trunk_policy(
    policy_input_lens_sums, action_lens_sums, 
    goal_len, n_hidden_layers, hidden_layer_size) # Change this to init a hardware conditioned if needed, 
# see apply_policy_hardware_conditioned notebook for usage

policy_network.load_state_dict(state_dict)


['llllll', 'lwllwl', 'wnwwnw']
fd_input_lens_sums, action_lens_sums, policy_input_lens_sums,fd_output_lens_sums: [45, 39, 21], [18, 16, 8], [41, 35, 17], [48, 42, 24]

attachments: [[1, 2, 3, 4, 5, 6], [0], [0], [0], [0], [0], [0]]
modules_types: [0, 1, 1, 1, 1, 1, 1]


<All keys matched successfully>

In [1]:
env.p.resetDebugVisualizerCamera(2.1,0,-65,[0,0,0.2],physicsClientId=env.physicsClient) 
# env.p.resetDebugVisualizerCamera(2.1,0,-89.999,[0,0,0.2],physicsClientId=env.physicsClient) 
env.reset_robot(urdf_name=urdf_name, randomize_start=False, start_xyyaw=[0,0,0])

# vid_fname='steer_vel.mp4'
# env.start_video_log(fileName=vid_fname)

# for step in range(150):
robot_button_mapping = [0,1,2,3,4,8,9,10]
n_buttons_to_check = len(robot_button_mapping)
from planning_utils import speed_scale_xy, speed_scale_yaw
# vxy_scale = 20*(20/240)*0.314/0.75
# vyaw_scale = 20*(20/240)*1.1/0.75
vxy_scale=speed_scale_xy
vyaw_scale=speed_scale_yaw

env_state = env.get_state()
last_states = [smm.to(device) for smm in to_tensors(env_state)]
# last_action = np.zeros(action_len)
tau_list = []
u_list = []
design_index = urdf_names.index(urdf_name)
buttons = np.zeros(12)
for step in range(10000):
    buttons = np.zeros(12)
    if USE_JOY:
        axes, buttons, povs = read(joy)
        axes = np.array(axes)
        axes[np.abs(axes)<0.01] = 0
        desired_xyyaw = np.array([-axes[1]*vxy_scale,
                                  -axes[0]*vxy_scale,
                                  -axes[2]*vyaw_scale])
        if buttons[9]==1 or buttons[8]==1:
            break
        
        if np.any(buttons[0:n_buttons_to_check]) or buttons[10] or buttons[11]:
            if np.any(buttons[0:n_buttons_to_check]):
                pressed = np.where(buttons[0:n_buttons_to_check])[0][0]
                urdf_name = urdf_names[robot_button_mapping[pressed]]
            else:
                ind = urdf_names.index(urdf_name)
                urdf_name = urdf_names[(ind+1) % len(urdf_names)]

                env.reset_robot(urdf_name=urdf_name, randomize_start=False, start_xyyaw=[0,0,0])
            attachments = env.attachments
            modules_types = env.modules_types
            n_modules = len(modules_types)
            env_state_init = env.get_state()
            module_state_len = []
            for s in env_state_init:
                module_state_len.append(len(s))
            state_len= np.sum(module_state_len)
            action_len = env.num_joints
            module_action_len = list(np.diff(env.action_indexes))
            module_sa_len = module_state_len+ module_action_len
            
            tau_list = []
            u_list = []
            last_states = [smm.to(device) for smm in to_tensors(env_state_init)]
            buttons = np.zeros(12)
            design_index = urdf_names.index(urdf_name)
            print(buttons)

    else:
        desired_xyyaw =  np.array([1,0, 0])

        
    chassis_yaw = env.pos_rpy[-1]
    vect1 = np.array([desired_xyyaw[0]*np.cos(chassis_yaw) - desired_xyyaw[1]*np.sin(chassis_yaw),
            desired_xyyaw[0]*np.sin(chassis_yaw) + desired_xyyaw[1]*np.cos(chassis_yaw),
            0] )
    vect2 = np.array([np.cos(desired_xyyaw[2]/2+chassis_yaw),
                      np.sin(desired_xyyaw[2]/2+chassis_yaw), 
                      0])*np.abs(desired_xyyaw[2])
    env.draw_body_arrows([vect1*0.5/vxy_scale, 
                          0.5*vect2/vyaw_scale],
                         [[0,0,0], [0,0,1]])

    
    env_state = env.get_state()
    states = [smm.to(device) for smm in to_tensors(env_state)]
    

    # heading and yaw here are wrt body frame
    goals = torch.tensor(desired_xyyaw, dtype=torch.float32, device=device).unsqueeze(0)

    inputs, goals = create_control_inputs(last_states, goals, rotate_goals = False)
    inputs = torch.cat(inputs, -1)

    with torch.no_grad():
        u_mean, u_var, tau_mean, tau_var = policy_network(
                inputs, goals, design_index)
        action = u_mean.squeeze().cpu().numpy()
        tau =  tau_mean.squeeze().cpu().numpy()
        tau_list.append(tau)
        u_list.append(action)
    
    env.step(action)
    

    
        
    if np.dot([0,0,1], env.z_axis)<0:
        env_yaw = env.pos_rpy[-1]
        env.reset_robot(urdf_name=urdf_name, randomize_start=True)
        env_state = env.get_state()
        states = [smm.to(device) for smm in to_tensors(env_state)]
        
    if np.sqrt(env.pos_xyz[0]**2+ env.pos_xyz[1]**2)>2:
        env_state = env.get_state()
        env_state[0][0:2] = 0 
        env.set_state(env_state)
    
    last_states = [s.clone() for s in states]



NameError: name 'env' is not defined